In [61]:
import pandas as pd
import numpy as np
from enum import Enum

# Constants - Market Data
DATE = 'date'
OPEN = 'open'
HIGH = 'high'
LOW = 'low'
CLOSE = 'close'
ADJ_CLOSE = 'adj_close'
VOLUME = 'volume'

# Constants - Position
BULLISH = 'bullish'
BEARISH = 'bearish'

# Constants - Strategy Events
OPEN_POSITION = 'open_position'
CLOSE_POSITION = 'close_position'
NO_EVENT = 'no_event'

# Technical Indicator
TI_CCI = 'CCI'

In [62]:
class DataCleaner():
    
    def __init__(self):
        self.dropna_method = 'all'
        self.is_fill_missing = True
    
    def clean(self, raw_data):
        raw_data = raw_data.dropna(how=self.dropna_method)
        if self.is_fill_missing:
            raw_data = self.fill_missing_date(raw_data)
        return raw_data
    
    @staticmethod
    def fill_missing_date(raw_data):
        cols = list(raw_data.columns)
        cols.remove(DATE)
        for col in cols:
            for i in raw_data.index:
                element = raw_data.at[i, col]
                if np.isnan(element) or element == 0:
                    missing_data_date = raw_data.at[i, DATE]
                    ref_data_date = raw_data.at[i-1, 'date']
                    raw_data.at[i, col] = raw_data.at[i-1, col]
                    print('[CLEAN] Copy data ({}, {}) to ({}, {}))'
                          .format(ref_data_date, col, missing_data_date, col))
        return raw_data

In [63]:
class TechIndicatorEngine():
    def __init__(self):
        pass
    
    def compute(self, data, ti_param):
        if self.extract_alphabet(ti_param.name) == TI_CCI:
            cci = CCI(ti_param)
            data = cci.compute(data)
        else:
            raise ValueError('[ERROR] Unknown technical indicator: {}'.format(ti_param.name))
        
        print('[TechIndicatorEngine] Finished computation of {}'.format(ti_param.name))
        return data
    
    @staticmethod
    def extract_alphabet(s):
        return ''.join(x for x in s if x.isalpha())

In [64]:
class TechIndicatorParam():
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        if self.__class__ != other.__class__:
            return False
        return self.__dict__ == other.__dict__


class CCIParam(TechIndicatorParam):
    def __init__(self, period, coeff):
        super().__init__(TI_CCI)
        self.period = period
        self.coeff = coeff


class CCI():
    
    def __init__(self, param):
        self.param = param
        
    def compute(self, data):
        
        # typical price
        data['typical'] = data[[HIGH, LOW, CLOSE]].apply(lambda row: self.typical_price(row), axis=1)
        
        # moving average of typical price
        data['MA'] = data['typical'].rolling(window=self.param.period).mean()
        
        # absolute difference between typical price and MA
        data['abs_diff'] = data[['typical', 'MA']].apply(lambda row: abs(row['typical'] - row['MA']), axis=1)
        
        # mean deviation
        data['MD'] = data['abs_diff'].rolling(window=self.param.period).mean()
        
        # CCI
        data[self.param.name] = data[['typical', 'MA', 'MD']].apply(lambda row: self.cci_formula(row), axis=1)
        
        # drop columns for intermediate steps
        data = data.drop(columns=['typical', 'MA', 'abs_diff', 'MD'])
        
        return data
        
    @staticmethod
    def typical_price(row):
        return (row[HIGH] + row[LOW] + row[CLOSE]) / 3
    
    def cci_formula(self, row):
        return (row['typical'] - row['MA']) / (self.param.coeff * row['MD'])

In [65]:
class Rule():
    def __init__(self, name, indicators_param, ref, f):
        self.name = name
        self.indicators_param = indicators_param
        self.ref = ref
        self.f = f

class OpenPositionParam():
    def __init__(self, rules):
        self.rules = rules  # rule: f(data, ref, self.entry_index, current_index) -> boolean

class ClosePositionParam():
    def __init__(self, rules):
        self.rules = rules  # rule : f(data, ref, self.entry_index, current_index) -> boolean


class Strategy():

    def __init__(self, name, open_position_param, close_position_param, position):
        self.name = name
        self.open_position = open_position_param
        self.close_position = close_position_param
        self.position = position
        self.have_position = False
        self.entry_index = None
        
    def check_event(self, data, current_index):       
        if not self.have_position:  # check for open position
            rule_triggered = self.check_rules(data, current_index, self.open_position.rules)
            if rule_triggered:
                self.have_position = True
                self.entry_index = current_index
                return OPEN_POSITION, rule_triggered
            
        else:  # check for close positioin
            rule_triggered = self.check_rules(data, current_index, self.close_position.rules)
            if rule_triggered:
                self.have_position = False
                self.entry_index = None
                return CLOSE_POSITION, rule_triggered
        
        return NO_EVENT, None
    
    def check_rules(self, data, current_index, rules):
        # logical OR on all the rules
        for rule in rules:
            if rule.f(data, rule.ref, self.entry_index, current_index):
                return rule.name
        return False

In [66]:
class BackTest():
    
    def __init__(self, raw_data, strategies):
        
        # class data
        self.data = raw_data
        self.trade_record = pd.DataFrame()
        self.report = pd.DataFrame()
        self.strategies = strategies
        self.all_ti_param = list()
        
        # service
        self.cleaner = DataCleaner()
        self.ti_engine = TechIndicatorEngine()
        
    def initialize(self):
        self.initialize_trade_record()
        self.initialize_report()
        self.initialize_all_ti_param()
        print('[BACKTEST] Initializated')
    
    def initialize_trade_record(self):
        self.trade_record = pd.DataFrame()
        self.trade_record['date'] = None
        self.trade_record['strategy_name'] = None
        self.trade_record['position'] = None
        self.trade_record['event'] = None
        self.trade_record['rule'] = None
        self.trade_record['price'] = None
        self.trade_record['trade_return'] = None
        
    def initialize_report(self):
        self.report = pd.DataFrame()
        self.report['strategy_name'] = None
        self.report['occurrence'] = None
        self.report['occ_profit'] = None
        self.report['occ_loss'] = None
        self.report['return_average'] = None
        self.report['return_std'] = None
        
    def initialize_all_ti_param(self):
        for strategy in self.strategies:
            for rule in (strategy.open_position.rules + strategy.close_position.rules):
                for param in rule.indicators_param:
                    
                    if param.name not in self.data.columns:
                        if not self.object_list_contains_object(self.all_ti_param, param):
                            
                            name = self.make_name([i.name for i in self.all_ti_param], param.name)
                            rule.ref[param.name] = name
                            param.name = name
                            self.all_ti_param.append(param)
                        
    @staticmethod
    def object_list_contains_object(obj_list, obj):
        for obj_temp in obj_list:
            if obj_temp == obj:
                return True
        return False
    
    @staticmethod
    def make_name(name_list, target):
        if target not in name_list:
            return target
        i = 2
        while(True):
            new_name = target + '_{}'.format(i)
            if new_name not in name_list:
                return new_name
            i += 1
    
    def data_preprocess(self):
        self.data = self.cleaner.clean(self.data)
        print('[BACKTEST] Finished data pre-process')
        
    def compute_technical_indicator(self):
        for param in self.all_ti_param:
            self.data = self.ti_engine.compute(self.data, param)
        print('[BACKTEST] Finished computation on technical indicators')
        
    def back_test(self):
        for strategy in self.strategies:
            for i in self.data.index:
                event, rule = strategy.check_event(self.data, i)
                if event is not NO_EVENT:
                    date = self.data.at[i, DATE]
                    price = self.data.at[i, CLOSE]
                    self.record_event(date, strategy, event, rule, price)
            print('[BACKTEST] Finished backtest on {}'.format(strategy.name))
        self.calculate_return()
        print('[BACKTEST] Finished all backtest')
    
    def record_event(self, date, strategy, event, rule, price):
        new_row = {'date': date,
                   'strategy_name': strategy.name,
                   'position': strategy.position,
                   'event': event,
                   'rule': rule,
                   'price': price}
        self.trade_record = self.trade_record.append(new_row, ignore_index=True)
        
    def calculate_return(self):
        self.trade_record.loc[self.trade_record['event'] == OPEN_POSITION, 'trade_return'] = np.NaN
        
        for strategy in self.strategies:
            
            strategy_trade_record = self.trade_record[self.trade_record['strategy_name'] == strategy.name]
            strategy_trade_record['original_id'] = strategy_trade_record.index
            strategy_trade_record = strategy_trade_record.reset_index()
            
            for i in strategy_trade_record.index:
                
                if strategy_trade_record.at[i, 'event'] == CLOSE_POSITION:
                    date = strategy_trade_record.at[i, DATE]
                    entry_price = strategy_trade_record.at[i-1, 'price']
                    exit_price = strategy_trade_record.at[i, 'price']
                    trade_return = self.compute_return(entry_price, exit_price, strategy.position)
                    original_id = strategy_trade_record.at[i, 'original_id']
                    self.trade_record.at[original_id, 'trade_return'] = trade_return
                    
    @staticmethod
    def compute_return(entry_price, exit_price, position):
        if position == BULLISH:
            return (exit_price - entry_price) / entry_price
        elif position == BEARISH:
            return -1 * (exit_price - entry_price) / entry_price
        else:
            raise ValueError('[ERROR] Unknown Position')
    
    def gen_report(self):
        for strategy in self.strategies:
            df = self.trade_record[self.trade_record['strategy_name'] == strategy.name]
            
            occurrence = df[df['event'] == CLOSE_POSITION].shape[0]
            occ_profit = df[df['trade_return'] > 0].shape[0]
            occ_loss = df[df['trade_return'] < 0].shape[0]
            return_average = df['trade_return'].mean()
            return_std = df['trade_return'].std()
            
            new_row = {'strategy_name': strategy.name,
                       'occurrence': occurrence,
                       'occ_profit': occ_profit,
                       'occ_loss': occ_loss,
                       'return_average': '{:.4%}'.format(return_average),
                       'return_std': return_std}
            self.report = self.report.append(new_row, ignore_index=True)
            
        self.gen_report_summary_row()
        print('[BACKTEST] Generated back test report')
        
    def gen_report_summary_row(self):
        new_row = {'strategy_name': 'summary',
                   'occurrence': self.report['occurrence'].sum(),
                   'occ_profit': self.report['occ_profit'].sum(),
                   'occ_loss': self.report['occ_loss'].sum(),
                   'return_average': '{:.4%}'.format(self.trade_record['trade_return'].mean()),
                   'return_std': self.trade_record['trade_return'].std()}
        self.report = self.report.append(new_row, ignore_index=True)
    
    def run(self):
        self.initialize()
        self.data_preprocess()
        self.compute_technical_indicator()
        self.back_test()
        self.gen_report()
        return

In [67]:
rule = Rule(name = 'basic', 
            indicators_param = [CCIParam(20, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < -198)
open_position_param = OpenPositionParam(rules=[rule])

rule = Rule(name = 'basic', 
            indicators_param = [CCIParam(20, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > 150)
close_position_param = ClosePositionParam(rules=[rule])

CCI_bull_strategy = Strategy('CCI_bull_strategy', open_position_param, close_position_param, BULLISH)


rule = Rule(name = 'basic', 
            indicators_param = [CCIParam(20, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] > 150)
open_position_param = OpenPositionParam(rules=[rule])

rule = Rule(name = 'basic', 
            indicators_param = [CCIParam(20, 0.015)],
            ref = {TI_CCI: TI_CCI},
            f = lambda data, ref, entry_i, i: data.at[i, ref[TI_CCI]] < -198)
close_position_param = ClosePositionParam(rules=[rule])

CCI_bear_strategy = Strategy('CCI_bear_strategy', open_position_param, close_position_param, BEARISH)


df = df = pd.read_csv('hsi.csv')
strategies = [CCI_bull_strategy, CCI_bear_strategy]
backtest = BackTest(df, strategies)
backtest.run()

[BACKTEST] Initializated
[CLEAN] Copy data (2020-10-12, open) to (2020-10-13, open))
[CLEAN] Copy data (2020-10-12, high) to (2020-10-13, high))
[CLEAN] Copy data (2020-10-12, low) to (2020-10-13, low))
[CLEAN] Copy data (2020-10-12, close) to (2020-10-13, close))
[CLEAN] Copy data (2020-10-12, adj_close) to (2020-10-13, adj_close))
[CLEAN] Copy data (2019-12-17, volume) to (2019-12-18, volume))
[CLEAN] Copy data (2020-02-25, volume) to (2020-02-26, volume))
[CLEAN] Copy data (2020-03-20, volume) to (2020-03-23, volume))
[CLEAN] Copy data (2020-05-05, volume) to (2020-05-06, volume))
[CLEAN] Copy data (2020-07-22, volume) to (2020-07-23, volume))
[CLEAN] Copy data (2020-10-12, volume) to (2020-10-13, volume))
[CLEAN] Copy data (2020-10-14, volume) to (2020-10-15, volume))
[BACKTEST] Finished data pre-process
[TechIndicatorEngine] Finished computation of CCI
[BACKTEST] Finished computation on technical indicators
[BACKTEST] Finished backtest on CCI_bull_strategy
[BACKTEST] Finished back

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
# backtest.all_ti_param[1].__dict__
backtest.strategies[0].open_position.rules[0].__dict__
backtest.strategies[0].close_position.rules[0].__dict__
backtest.data

,date,open,high,low,close,adj_close,volume,CCI
0,2019-10-15,26609.13086,26609.13086,26433.42969,26503.92969,26503.92969,1.569212e+09,NaN
1,2019-10-16,26553.90039,26698.56055,26432.42969,26664.27930,26664.27930,1.689389e+09,NaN
2,2019-10-17,26809.03906,26956.82031,26729.50977,26848.49023,26848.49023,1.403839e+09,NaN
3,2019-10-18,26910.55078,26985.08008,26650.13086,26719.58008,26719.58008,1.541856e+09,NaN
4,2019-10-21,26715.22070,26818.43945,26658.32031,26725.67969,26725.67969,1.045152e+09,NaN
...,...,...,...,...,...,...,...,...
245,2020-10-09,24343.81055,24343.81055,24059.28906,24119.13086,24119.13086,1.358970e+09,16.642586
246,2020-10-12,24217.25000,24702.81055,24196.80078,24649.67969,24649.67969,2.430404e+09,57.312498
247,2020-10-13,24217.25000,24702.81055,24196.80078,24649.67969,24649.67969,2.430404e+09,57.540346
248,2020-10-14,24808.84961,24808.84961,24498.93945,24667.08984,24667.08984,2.550043e+09,72.962800


In [69]:
backtest.trade_record

,date,strategy_name,position,event,rule,price,trade_return
0,2020-03-09,CCI_bull_strategy,bullish,open_position,basic,25040.46094,NaN
1,2020-07-06,CCI_bull_strategy,bullish,close_position,basic,26339.16016,0.051864
2,2019-12-13,CCI_bear_strategy,bearish,open_position,basic,27687.75977,NaN
3,2020-03-09,CCI_bear_strategy,bearish,close_position,basic,25040.46094,0.095613
4,2020-07-06,CCI_bear_strategy,bearish,open_position,basic,26339.16016,NaN


In [70]:
backtest.report

,strategy_name,occurrence,occ_profit,occ_loss,return_average,return_std
0,CCI_bull_strategy,1,1,0,5.1864%,NaN
1,CCI_bear_strategy,1,1,0,9.5613%,NaN
2,summary,2,2,0,7.3738%,0.030935


In [25]:
TI.CCI.value

'CCI'